# <center>我整个重写一遍吧

## 所有用到的环境

In [1]:
import functools;
import time;
import types;
import typing;

In [2]:
import torch;

## 代码兼容性评估

### Some General Functions

In [3]:
def getAPIName(api):
    # for function type
    if isinstance(api, types.FunctionType) or isinstance(api, types.BuiltinFunctionType):
        apiName = api.__module__ + "." + api.__name__;
        return apiName;
    # for class type
    elif isinstance(api, type):
        apiName = api.__module__;
        if api.__name__[-len(api.__name__)+1:] == apiName[-len(api.__name__)+1:]:
            apiName = apiName[:-len(api.__name__)] + api.__name__;
        apiName = apiName + "." + api.__name__;
        return apiName;
    # for module type
    elif isinstance(api, types.ModuleType):
        return api.__name__;
    # 
    elif api.__module__ == "typing":
        apiName = api.__module__ + "." + api.__name__;
        return apiName;
    else:
        return str(type(api));
getAPIName(torch.optim.Adam),getAPIName(torch.mul)

('torch.optim.Adam.Adam', 'torch.mul')

In [4]:
def isFromModule(api: types.FunctionType, module: str):
    return getAPIName(api).startswith(module);
isFromModule(torch._utils, "torch"),type(torch.optim)

(True, module)

In [5]:
# def getAttributes(module):
#     return [i for i in dir(module) if not i.startswith("__") and callable(getattr(module, i)) or isinstance(getattr(module, i), types.ModuleType)];
# getAttributes(torch.optim)
def getAttributes(module):
    attributes = [];
    for i in module.__dict__.keys():
        if not i.startswith("__"):
            try:
                attr = getattr(module, i);
                if callable(attr) or isinstance(attr, types.ModuleType):
                    attributes.append(i);
            except AttributeError:
                continue;
            except Exception as e:
                print(f"Error accessing attribute {i} of {module}: {e}");
                continue;
    return attributes;
getAttributes(torch.optim)

['Adadelta',
 'Adagrad',
 'Adam',
 'AdamW',
 '_functional',
 'SparseAdam',
 'Adamax',
 'ASGD',
 'SGD',
 'RAdam',
 'Rprop',
 'RMSprop',
 'Optimizer',
 'NAdam',
 'LBFGS',
 'lr_scheduler',
 'swa_utils',
 '_multi_tensor']

In [6]:
getAttributes(torch.optim.lr_scheduler.Counter)

['total',
 'most_common',
 'elements',
 'fromkeys',
 'update',
 'subtract',
 'copy',
 '_keep_positive']

In [7]:
torch.tensor.__module__,getAPIName(torch.optim),torch._C.__name__

('torch', 'torch.optim', 'torch._C')

In [8]:
def isDecorated(obj: (types.FunctionType, types.ModuleType, type)):
    return getattr(obj, "_isDecorated", False);
isDecorated(torch.tensor)

False

### Some General Decorators Patterns

#### 需要的环境

In [9]:
import functools;
import time;
import types;
import warnings;
import sys;

#### TimerDecorator

In [10]:
def TimerDecorator(func: types.FunctionType):
    """
    **Description**
    A running timer for a function.
    
    **params**
    func(String): the function to be timed.
    
    **returns**
    wrapper: a timer decorated function.
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        startTime = time.perf_counter_ns();
        result = func(*args, **kwargs);
        endTime = time.perf_counter_ns();
        costTime = (endTime - startTime) / 1000 / 1000;
        timeLog = f"{func.__name__}() cost {costTime} ms";
        # print(timeLog);
        return result, str(startTime), costTime;
    return wrapper;

Test Cases

In [11]:
def tst():
    t = 1;
    for i in range(5):
        t *= (i+1);
    return t;

TimerDecorator(torch.matmul)(torch.tensor([[1.0,5.0],[3.07,7.29]]), torch.tensor([[6.08,9.05],[3.4,2.8]])), "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%",\
TimerDecorator(tst)()

((tensor([[23.0800, 23.0500],
          [43.4516, 48.1955]]),
  '102845888299300',
  0.1584),
 '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 (120, '102845888487100', 0.0024))

#### APIDecorator

In [12]:
def APIDecorator(func: str, module: str=None):
    """
    **Description**
    A API usage recorder for a function.
    
    **params**
    func(String): the function to be recorded.
    
    **returns**
    wrapper: a API usage recored function.
    """
    apiName = getAPIName(func)
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        apiName = getAPIName(func)
        result, startTimestamp, costTime= TimerDecorator(func)(*args, **kwargs);
        print(f"{apiName} starts from {startTimestamp} costs {costTime}ms.");
        return result, apiName, startTimestamp, costTime;
    if module == None:
        return wrapper;
    elif isFromModule(func, module):
        return wrapper;
    else:
        # raise ValueError(f"the function `{apiName}` is not from module `{module}`");
        return func;

Test Cases

In [13]:
APIDecorator(torch.normal, "types")(torch.tensor([0,0.01])), "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%", \
APIDecorator(torch.randn, "torch")([2,3]), "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%", \
APIDecorator(torch.mul)(torch.tensor([2.5,0,1,3]), torch.tensor([2,0.9,2,4]))

torch.randn starts from 102845902581700 costs 0.0497ms.
torch.mul starts from 102845902730000 costs 0.1343ms.


(tensor([-1.2677, -0.6425]),
 '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 (tensor([[1.5157, 1.7160, 0.5315],
          [1.6908, 0.0873, 0.4804]]),
  'torch.randn',
  '102845902581700',
  0.0497),
 '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 (tensor([ 5.,  0.,  2., 12.]), 'torch.mul', '102845902730000', 0.1343))

In [14]:
torch.optim.__name__

'torch.optim'

### Class TorchWrapper

#### 需要的环境

In [15]:
import functools;
import time;
import types;
import json;
import os;
import operator;
import pandas as pd;
import torch;
import sys;

#### TorchWrapper

```
*****************************
The Structure For callRecords
*****************************
callRecords
│
├── API_1
│   │
│   ├── TotalTime(ms): 150.0
│   │
│   ├── 1
│   │   ├── detailedAPIName: 
│   │   ├── StartTimestamp: 1625150800123456789
│   │   ├── CostTime(ms): 50.0
│   │   └── Arguments: (arg1, arg2, ...)
│   │
│   ├── 2
│   │   ├── detailedAPIName: 
│       ├── StartTimestamp: 1625150860123456789
│       ├── CostTime(ms): 100.0
│       └── Arguments: (arg1, arg2, ...)
│
├── API_2
│   │
│   ├── TotalTime(ms): 200.0
│   │
│   ├── 1
│   │   ├── detailedAPIName: 
│       ├── StartTimestamp: 1625150900123456789
│       ├── CostTime(ms): 200.0
│       └── Arguments: (arg1, arg2, ...)
```

In [16]:
class TorchWrapper:
    """
    A wrapper class for torch functions and modules to record API call details.
    """
        
    """
    ********************
    Initializing Section
    ********************
    """

    GOAL_MODULE = "torch";
    DEFAULT_PATH = "./results"
    DEFAULT_FORMAT = "csv";
    DEFAULT_NAME_SPEC = "timestamp";
    SUPPORTED_FORMATS = ["json", "csv", "html", "xlsx"];
    SUPPORTED_NAME_SPEC = ["timestamp", "datetime", "serial"];

    class ConfigKey:
        GOAL_MODULE = "goal_module";
        OUT_DIR = "out_dir";
        FORMAT = "format";
        FILE_MAX_SIZE = "file_max_size";
        FILE_NAME_SPEC = "file_name_spec";

    class CallRecordKey:
        API_NAME = "APIName";

        class ResultKey:
            TOTAL_TIME = "TotalTime(ms)";
            CALL_NUMBER = "CallNumber";
            START_TIMESTAMP = "StartTimestamp";
            COST_TIME = "CostTime(ms)";
            ARGUMENTS = "Arguments";

    def __init__(self, config: dict):
        """
        Initialize the TorchWrapper with the provided configuration.
        
        **params**
        config (dict): The configuration dictionary.
        """
        self.callRecords = {};
        self.config = self.parseConfig(config);
        print(f"your wrapper config: {self.getConfig()}");
        

    def parseConfig(self, config: dict) -> dict:
        """
        Parse and validate the configuration dictionary.
        
        **params**
        config (dict): The configuration dictionary.
        
        **returns**
        dict: The parsed and validated configuration.
        
        **raises**
        ValueError: If any required configuration is invalid.
        """
        if TorchWrapper.ConfigKey.OUT_DIR not in config:
            raise ValueError("Output directory is required.");
        assert isinstance(config[TorchWrapper.ConfigKey.OUT_DIR], str);

        if TorchWrapper.ConfigKey.FORMAT in config:
            assert isinstance(config[TorchWrapper.ConfigKey.FORMAT], str);
            format = config[TorchWrapper.ConfigKey.FORMAT];
            if format not in TorchWrapper.SUPPORTED_FORMATS:
                raise ValueError(f"Unsupported format {format} for saving result");
        else:
            config[TorchWrapper.ConfigKey.FORMAT] = TorchWrapper.DEFAULT_FORMAT;

        if TorchWrapper.ConfigKey.FILE_MAX_SIZE in config:
            assert isinstance(config[TorchWrapper.ConfigKey.FILE_MAX_SIZE], str);
            if config[TorchWrapper.ConfigKey.FILE_MAX_SIZE][-2:] not in ["KB", "MB", "GB"]:
                raise ValueError("maxSize should be defined in the style of `myInt`KB/MB/GB");

        if TorchWrapper.ConfigKey.FILE_NAME_SPEC in config:
            assert isinstance(config[TorchWrapper.ConfigKey.FILE_NAME_SPEC], str);
            name_spec = config[TorchWrapper.ConfigKey.FILE_NAME_SPEC];
            if name_spec not in TorchWrapper.SUPPORTED_NAME_SPEC:
                raise ValueError(f"Unsupported file name spec {name_spec}");
        else:
            config[TorchWrapper.ConfigKey.FILE_NAME_SPEC] = TorchWrapper.DEFAULT_NAME_SPEC;

        return config;

    def getConfig(self) -> dict:
        return self.config;
    
    """
    ******************
    decorating Section
    ******************
    """

    def CountDecorator(self, func: types.FunctionType) -> types.FunctionType:
        """
        A decorator that counts the calls of a function and records it.
        
        **params**
        func (types.FunctionType): The function to be recorded calling times.
        
        **returns**
        types.FunctionType: A function that has been counted calling times.
        """
        funcName = getAPIName(func);
        # print(f"decorating function {funcName}");
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            record = {
                TorchWrapper.CallRecordKey.ResultKey.CALL_NUMBER: None,
                TorchWrapper.CallRecordKey.ResultKey.START_TIMESTAMP: None,
                TorchWrapper.CallRecordKey.ResultKey.COST_TIME: None,
                TorchWrapper.CallRecordKey.ResultKey.ARGUMENTS: (args, kwargs)
            };

            result, apiName, startTimestamp, costTime = APIDecorator(func)(*args, **kwargs);
            if apiName in self.callRecords:
                callCount = len(self.callRecords[apiName].keys());
                totalTime = self.callRecords[apiName][TorchWrapper.CallRecordKey.ResultKey.TOTAL_TIME];
            else:
                callCount = 1;
                totalTime = 0.0;

            record[TorchWrapper.CallRecordKey.ResultKey.START_TIMESTAMP] = startTimestamp;
            record[TorchWrapper.CallRecordKey.ResultKey.COST_TIME] = costTime;
            totalTime += costTime;

            self.callRecords.setdefault(apiName, {})[TorchWrapper.CallRecordKey.ResultKey.TOTAL_TIME] = totalTime;
            self.callRecords[apiName][callCount] = record;

            return result;
        wrapper._isDecorated = True;
        # print(f"{funcName} decorated.");
        return wrapper;
    
    def decorateFunction(self, module: (types.ModuleType, type), name: str, func: (types.FunctionType, types.BuiltinFunctionType)):
        setattr(module, name, self.CountDecorator(func));

    def decorateClass(self, cls: type):
        """
        Decorates all methods of a class with CountDecorator and records the API names.

        **params**
        cls (type): The class whose methods are to be decorated with CountDecorator.

        **returns**
        None
        """
        clsName = cls.__module__ + "." + cls.__name__;
        if isFromModule(cls, TorchWrapper.GOAL_MODULE):
            assert isinstance(cls, type), f"`{cls}` must be a class.";

            if isDecorated(cls):
                return ; # Stop decoration don't need specific returns
            print(f"\t[Class] Class `{clsName}` is not decorated, decorate it.");

            attributes = getAttributes(cls);
            if attributes:
                for name in attributes:
                    try:
                        cls._isDecorated = True;
                        cattr = getattr(cls, name);
                        apiName = getAPIName(cattr);
                        if isinstance(cattr, (types.FunctionType, types.BuiltinFunctionType)) and isFromModule(cattr, TorchWrapper.GOAL_MODULE) and not isDecorated(cattr):
                            print(f"\t\t[Method] Method `{name}` hasn't been decorated, \n\t\tdecorating `{name}`.");
                            self.decorateFunction(cls, name, cattr);
                            print(f"\t\tMethod `{name}` has been decorated.");
                        elif isinstance(cattr, (type, types.ModuleType)) and isFromModule(cattr, TorchWrapper.GOAL_MODULE) and not isDecorated(cattr):
                            self.decorateClass(cattr);

                    except TypeError as e:
                        if "immutable type" in str(e):
                            raise TypeError(f"\t[IMU]`{name}` is an immutable type, out.") from e;
                            continue;
                        raise NameError(f"[decorateClass]The attribute that cause TypeError: `{name}`") from e;
        else:
            print(f"\t[EMN]Class `{moduleName}` is a external class or subclass, not inside `{TorchWrapper.GOAL_MODULE}`, out.");
            return;


    def decorateModule(self, module: types.ModuleType):
        """
        Decorates all functions and classes of a module with CountDecorator.
        
        **params**
        module (types.ModuleType): The module to be decorated.
        
        **returns**
        None
        """
        moduleName = getAPIName(module);
        if isFromModule(module, TorchWrapper.GOAL_MODULE):
            assert isinstance(module, types.ModuleType), f"`{module}` must be a module.";
            if isDecorated(module):
                # print(f"module {moduleName} has been decorated, out.");
                return; # Stop the decoration, no specific return.
            module._isDecorated = True;
            for name in getAttributes(module):
                try:
                    mattr = getattr(module, name);
                    apiName = getAPIName(mattr);
                    if isinstance(mattr, types.ModuleType) and isFromModule(mattr, TorchWrapper.GOAL_MODULE):
                        print(f"\t[SubModule]Submodule `{name}` of {moduleName} hasn't been decorated, \n\tdecorating {name}.");
                        self.decorateModule(mattr);
                        if isDecorated(mattr):
                            print(f"\t[SubModule]Submodule `{name}` has been decorated.");
                    elif isinstance(mattr, types.FunctionType) and isFromModule(mattr, TorchWrapper.GOAL_MODULE) and not isDecorated(mattr):
                        print(f"\t[Operator]Operator `{apiName}` hasn't been decorated, \n\tdecorating {name}.");
                        self.decorateFunction(module, name, mattr);
                        if isDecorated(mattr):
                            print(f"\t[Operator]Opertator `{apiName}` has been decorated.");
                    elif isinstance(mattr, type) and isFromModule(mattr, TorchWrapper.GOAL_MODULE) and not isDecorated(mattr):
                        print(f"\t[SubClass]SubClass `{name}` of {moduleName} hasn't been decorated, \n\tdecorating {apiName}.");
                        self.decorateClass(mattr);
                        if isDecorated(mattr):
                            print(f"\t[SubClass]SubClass `{name}` has been decorated.");
                    elif isDecorated(mattr):
                        print(f"\t[Decorated]Attribute `{name}` of {moduleName} has been decorated, out.");
                    elif not isFromModule(mattr, TorchWrapper.GOAL_MODULE):
                        print(f"[EXTATTR]`{apiName}` is a external attribute, not inside `{TorchWrapper.GOAL_MODULE}`, out.");
                    else:
                        print(f"\t[SkipDecoration]Attribute `{name}` of {moduleName} skiped decoration, for unknow reason.");
                except Exception as e:
                    raise NameError(f"[decorateModule]The attribute that cause Error: `{name}`") from e;
                    print(f"The attribute that cause Error: `{name}`");
                    continue;
                    

        else:
            print(f"module {moduleName} is a external module, not in {TorchWrapper.GOAL_MODULE}, out.");
    
    """
    ******************
    Processing Section
    ******************
    """
    
    def getFileMaxSize(self, config: dict) -> int:
        """
        Parse the max file size limit from the configuration.
        
        **params**
        config (dict): The configuration dictionary.
        
        **returns**
        int: The max file size in bytes.
        
        **raises**
        ValueError: If the max size format is invalid.
        """
        if TorchWrapper.ConfigKey.FILE_MAX_SIZE in config:
            maxSize = config[TorchWrapper.ConfigKey.FILE_MAX_SIZE];
            if maxSize.endswith("KB"):
                return int(maxSize[:-2]) * 1024;
            elif maxSize.endswith("MB"):
                return int(maxSize[:-2]) * (1024 ** 2);
            elif maxSize.endswith("GB"):
                return int(maxSize[:-2]) * (1024 ** 3);
            else:
                raise ValueError("maxSize should be defined in the style of `myInt`KB/MB/GB(e.g.'64MB')");
        return None;

    def getFileNameSuffix(self) -> str:
        """
        Get the file name suffix based on the configuration.
        
        **returns**
        str: The file name suffix.
        
        **raises**
        NotImplementedError: If the file name specification is not implemented.
        """
        file_name_spec = self.config[TorchWrapper.ConfigKey.FILE_NAME_SPEC];
        if file_name_spec == "timestamp":
            return str(time.time_ns());
        elif file_name_spec == "datetime":
            return time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime());
        elif file_name_spec == "serial":
            raise NotImplementedError("Serial file name specification is not implemented.");
        return ""

    def setPath(self, path: str) -> str:
        """
        Prepare the result saving directory.
        
        **params**
        path (str): The directory path.
        
        **returns**
        str: The validated directory path.
        
        **raises**
        ValueError: If the path is not a directory.
        """
        if os.path.exists(path):
            if not os.path.isdir(path):
                raise ValueError(f"Path {path} is not a directory");
        else:
            os.makedirs(path);
        return path;

    def getFileName(self, config: dict) -> str:
        """
        Get the file name for saving the results.
        
        **returns**
        str: The file name.
        """
        suffix = self.getFileNameSuffix();
        return f"TorchWrapper_Result_{suffix}";

    def getDFFormattedCallRecords(self) -> pd.DataFrame:
        """
        Formats the call records as a pandas DataFrame.
        
        **returns**
        pd.DataFrame: The formatted call records.
        """
        records = []
        for apiName, calls in self.callRecords.items():
            totalTime = calls.pop(TorchWrapper.CallRecordKey.ResultKey.TOTAL_TIME, 0);
            for callNumber, call in calls.items():
                record = {
                    TorchWrapper.CallRecordKey.API_NAME: apiName,
                    TorchWrapper.CallRecordKey.ResultKey.TOTAL_TIME: totalTime,
                    TorchWrapper.CallRecordKey.ResultKey.CALL_NUMBER: callNumber,
                    TorchWrapper.CallRecordKey.ResultKey.START_TIMESTAMP: call[TorchWrapper.CallRecordKey.ResultKey.START_TIMESTAMP],
                    TorchWrapper.CallRecordKey.ResultKey.COST_TIME: call[TorchWrapper.CallRecordKey.ResultKey.COST_TIME],
                    TorchWrapper.CallRecordKey.ResultKey.ARGUMENTS: call[TorchWrapper.CallRecordKey.ResultKey.ARGUMENTS]
                };
                records.append(record);
        return pd.DataFrame(records);

    def saveRecords(self, config: dict):
        """
        Save the call records to a file based on the configuration.
        
        **params**
        config (dict): The configuration dictionary.
        """
        def saveToJson(data: pd.DataFrame, path: str, fileName: str):
            data.to_json(f"{path}/{fileName}.json", orient='records', lines=True);
            return f"{path}/{fileName}.json";

        def saveToCSV(data: pd.DataFrame, path: str, fileName: str):
            data.to_csv(f"{path}/{fileName}.csv", index=False);

        def saveToExcel(data: pd.DataFrame, path: str, fileName: str):
            data.to_excel(f"{path}/{fileName}.xlsx", index=False);

        def saveToHTML(data: pd.DataFrame, path: str, fileName: str):
            data.to_html(f"{path}/{fileName}.html", index=False);

        fileName = self.getFileName(config);
        data = self.getDFFormattedCallRecords();
        outputPath = self.setPath(config[TorchWrapper.ConfigKey.OUT_DIR]);
        if config[TorchWrapper.ConfigKey.FORMAT] == "json":
            saveToJson(data, outputPath, fileName);
            return f"{outputPath}/{fileName}.json";
        elif config[TorchWrapper.ConfigKey.FORMAT] == "csv":
            saveToCSV(data, outputPath, fileName);
            return f"{outputPath}/{fileName}.csv";
        elif config[TorchWrapper.ConfigKey.FORMAT] == "xlsx":
            saveToExcel(data, outputPath, fileName);
            return f"{outputPath}/{fileName}.xlsx";
        elif config[TorchWrapper.ConfigKey.FORMAT] == "html":
            saveToHTML(data, outputPath, fileName);
            return f"{path}/{fileName}.html";
            
    """
    **************
    Usable Section
    **************
    """

    def start(self, func: types.FunctionType):
        """
        Starts the wrapping and recording process.
        
        **params**
        func (types.FunctionType): The function to be executed and recorded.
        
        **returns**
        Any: The result of the executed function.
        callRecords: the DataFrameFormatted callRecords.
        
        **raises**
        ValueError: If there is an error executing the goal function.
        """
        
        # decorate the module to be evaluated
        print(f"Starts decorating torch module.");
        self.decorateModule(torch);
        print("torch module decorating complete.");
        
        # run the codes to be evaluated.
        try:
            print(f"Starts evaluating {func.__name__}");
            result = func();

        except Exception as e:
            print(f"Error executing the function: {e}");
            raise ValueError("Error executing the function, check your code first.") from e;
            
        # saving the results
        print("start saving results.");
        path = self.saveRecords(self.config);
        print(f"results file saved to `{path}`");
        return result, self.getDFFormattedCallRecords();

In [17]:
config = {
    "out_dir": "./output",
    "file_max_size": "10MB",
    "file_name_spec": "timestamp"
};
wrapper = TorchWrapper(config)

your wrapper config: {'out_dir': './output', 'file_max_size': '10MB', 'file_name_spec': 'timestamp', 'format': 'csv'}


In [18]:
def myCode():
    a = torch.randn(1, 3);
    b = torch.randn(1, 3);
    c = a + b;
    return c;

In [19]:
config = {
    "out_dir": "./output",
    "format": "csv",
    "file_max_size": "10MB",
    "file_name_spec": "timestamp"
};
wrapper = TorchWrapper(config);

wrapper.start(myCode);

your wrapper config: {'out_dir': './output', 'format': 'csv', 'file_max_size': '10MB', 'file_name_spec': 'timestamp'}
Starts decorating torch module.
[EXTATTR]`math` is a external attribute, not inside `torch`, out.
[EXTATTR]`os` is a external attribute, not inside `torch`, out.
[EXTATTR]`sys` is a external attribute, not inside `torch`, out.
[EXTATTR]`platform` is a external attribute, not inside `torch`, out.
[EXTATTR]`textwrap` is a external attribute, not inside `torch`, out.
[EXTATTR]`ctypes` is a external attribute, not inside `torch`, out.
[EXTATTR]`inspect` is a external attribute, not inside `torch`, out.
	[Operator]Operator `torch._running_with_deploy` hasn't been decorated, 
	decorating _running_with_deploy.
	[SubModule]Submodule `_utils` of torch hasn't been decorated, 
	decorating _utils.
[EXTATTR]`copyreg` is a external attribute, not inside `torch`, out.
[EXTATTR]`functools` is a external attribute, not inside `torch`, out.
[EXTATTR]`sys` is a external attribute, not ins

NameError: [decorateModule]The attribute that cause Error: `_C`

In [ ]:
torch.storage

In [ ]:
dir(torch).index("_assert")

In [ ]:
len(dir(torch))

In [ ]:
torch._assert.__class__

In [ ]:
torch._isDecorated

In [20]:
for name in getAttributes(torch):
    try:
        attr = getattr(torch, name)
        print(f"{getAPIName(attr):<70}{name:<65}{isFromModule(attr, 'torch')}");
    except Exception as e:
        raise NameError(f"The attribution that cause error is {name}") from e

math                                                                  math                                                             False
os                                                                    os                                                               False
sys                                                                   sys                                                              False
platform                                                              platform                                                         False
textwrap                                                              textwrap                                                         False
ctypes                                                                ctypes                                                           False
inspect                                                               inspect                                                          False
torch._runnin

In [23]:
type(torch._import_dotted_name)

function

In [ ]:
isFromModule(torch._utils, "torch")
getAPIName(torch.quantized_lstm)

In [ ]:
import torch

In [ ]:
getAttributes(torch).index("matmul")

In [ ]:
def add(*args):
    return sum(args);
add(2,7,4,6,9,5,3,3,8,7)

In [ ]:
isDecorated(torch._C.Block),isinstance(torch._C.Block, type)

In [ ]:
getAttributes(torch).index("list_backends")

In [ ]:
torch._utils.__name__

In [ ]:
torch.Callable.__module__

In [ ]:
torch.builtins.__name__

In [ ]:
torch.optim.__name__

In [ ]:
isinstance(torch.Size, type),torch.Size.__name__

In [28]:
import time
for na in getAttributes(time):
    attr = getattr(time, na);
    print(getAPIName(attr))

time.time
time.time_ns
time.sleep
time.gmtime
time.localtime
time.asctime
time.ctime
time.mktime
time.strftime
time.strptime
time.monotonic
time.monotonic_ns
time.process_time
time.process_time_ns
time.thread_time
time.thread_time_ns
time.perf_counter
time.perf_counter_ns
time.get_clock_info
time.struct_time
